#### Requirements 

In [ ]:
# !pip install tensorflow 
# !pip install numpy 
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn
# !pip install seaborn


#### GPU usage for tf

In [ ]:
!nvidia-smi # veriying if NVIDEA drive and CUDA runtime loads 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

# 1) Ver todas as GPUs
gpus = tf.config.list_physical_devices("GPU")
print("GPUs detectadas:", gpus)

if gpus:
    # 2) (Opcional) limitar a visão só à primeira GPU
    tf.config.set_visible_devices(gpus[0], "GPU")

    # 3) (Recomendado) liberar memória sob demanda
    tf.config.experimental.set_memory_growth(gpus[0], True)

#### Other libs

In [ ]:
# LIBS

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import TimeSeriesSplit
import keras

register_matplotlib_converters()
sns.set_style("darkgrid")

plt.rc("figure", figsize=(16, 6))
plt.rc("font", size=13)

from matplotlib.pyplot import figure

figure(figsize = (16, 6), dpi = 100)

#### Model improvement

In [ ]:
#gridsearch
def grid_search_cv(modelo, units, X_train, learning_rates, y_train, epochs_list, batch_sizes, patiences, model_name):
    best_loss = float('inf')
    best_params = {}
    for lr in learning_rates:
        for epochs in epochs_list:
            for batch_size in batch_sizes:
                for patience in patiences:
                    model = modelo(units, X_train, lr)
                    histories = fit_model_with_cross_validation(model, X_train, y_train, model_name, patience, epochs, batch_size)
                    mean_history = calculate_mean_history(histories)
                    val_loss = min(mean_history['val_loss'])
                    print("Val Loss: ", val_loss, "learning rate: ", lr, "epochs: ",  epochs, "batch_size: " , batch_size, "patience: ", patience)
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_params = {'learning_rate': lr, 'epochs': epochs, 'batch_size': batch_size, 'patience': patience} 
    print('O modelo '+model_name+ ' tem como melhores parametros os seguintes: learning_rate '+ str(best_params['learning_rate'])+' epochs: '+ str(best_params['epochs'])+' batch_size: '+ str(best_params['batch_size'])+ ' patience: '+ str(best_params['patience']))
    return best_params

#validação cruzada
def fit_model_with_cross_validation(model, xtrain, ytrain, model_name, patience, epochs, batch_size):
    tscv = TimeSeriesSplit(n_splits=5)
    fold = 1
    histories = []
    for train_index, val_index in tscv.split(xtrain):
        x_train_fold, x_val_fold = xtrain[train_index], xtrain[val_index]
        y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, min_delta=1e-5)
        history = model.fit(x_train_fold, y_train_fold, epochs=epochs, validation_data=(x_val_fold, y_val_fold), batch_size=batch_size, callbacks=[early_stop], verbose=1)
        print('\n\nTREINAMENTO - Fold', fold, 'do modelo:', model_name)
        histories.append(history)
        fold += 1   
    return histories 

# calcula a media das metricas obtidas nos historys - validação cruzada
def calculate_mean_history(histories):
    mean_history = {'loss': [], 'root_mean_squared_error': [], 'val_loss': [], 'val_root_mean_squared_error': []}
    for fold_history in histories:
        for key in mean_history.keys():
            mean_history[key].append(fold_history.history[key])
    for key, values in mean_history.items():
        max_len = max(len(val) for val in values)
        for i in range(len(values)):
            if len(values[i]) < max_len: #caso em que nao se treina todas as epocas (patience)
                values[i] += [values[i][-1]] * (max_len - len(values[i])) #completa o restante da lista com o ultimo valor obtido
    for key, values in mean_history.items():
        mean_history[key] = [sum(vals) / len(vals) for vals in zip(*values)]
    
    return mean_history


#### LSTM construction

In [ ]:
# Create input dataset
# The input shape should be [samples, time steps, features
def create_dataset (X, look_back = 3):
    Xs, ys = [], []
    
    for i in range(len(X)-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])
        
    return np.array(Xs), np.array(ys)

# Create LSTM model
def create_lstm(units, train, learning_rate): 
    model = Sequential() 
    # Old Config
    model.add(LSTM(units = units, return_sequences = True, input_shape = [train.shape[1], train.shape[2]]))
    model.add(LSTM(units = units)) 
    # model.add(Dropout(0.2))
    model.add(Dense(1))
    # model.compile(loss=MeanSquaredError(), optimizer = Adam(learning_rate=learning_rate), metrics=[RootMeanSquaredError()])
    model.compile(
        loss=Huber(delta=0.25),  # delta define quando a perda muda de quadrática para linear
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[RootMeanSquaredError()]
    )
    
    return model

#treinamento do modelo
def fit_model(model, xtrain, ytrain, model_name, patience, epochs, batch_size ):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience, restore_best_weights=True)
    history = model.fit(xtrain, ytrain, epochs = epochs, validation_split = 0.2, batch_size = batch_size, shuffle = True, callbacks=[early_stop]) 
    print('\n\nTREINAMENTO: ' + model_name)
    return history

# Make prediction
def prediction(model, xtest, ytest, myscaler, model_name, link): 
    prediction = model.predict(xtest) 
    prediction = myscaler.inverse_transform(prediction) 
    # dataframe_prediction = pd.DataFrame(data={'Predições':prediction.flatten()})
    dataframe_prediction = pd.DataFrame(data={'Prediction':prediction.flatten(), 'Test':ytest.flatten()})
    #save_path = os.path.join('..', '..', 'predicoes', f'prediction {model_name} {link}.csv') 
    save_path = os.path.join('..', '..', 'results', 'bi-lstm', 'forecast', f'prediction {model_name} {link}.csv') 
    dataframe_prediction.to_csv(save_path)
    return prediction


# Calculate MAE and RMSE
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    nrmse = rmse/ np.max(actual)
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Normalized Root Mean Square Error: {:.4f}%'.format(nrmse*100))
    print('')

    return rmse, mae, nrmse, model_name

# ===================================================================================
# NOVA FUNÇÃO: valida métrica apenas em pontos com dado real (mask == True)
# ===================================================================================
def validate_missing_data_prediction(predictions, actual, mask, model_name):
    """
    Calcula RMSE, MAE e NRMSE APENAS nos pontos onde há dado real (mask==True).

    Parameters
    ----------
    predictions : np.ndarray  (shape: [N, 1] ou [N])
        Vetor de predições PURO, sem mistura com pontos reais.
    actual      : np.ndarray  (shape: [N, 1] ou [N])
        Vetor de valores reais correspondentes.
    mask        : np.ndarray  (shape: [N], dtype=bool)
        True  -> ponto com dado real
        False -> ponto ausente (gap) onde não devemos avaliar.
    model_name  : str
        Identificador do modelo (ex.: 'LSTM')

    Returns
    -------
    (rmse, mae, nrmse, model_name)
    """
    # Garante formato 1-D
    predictions = predictions.flatten()
    actual      = actual.flatten()
    mask        = mask.astype(bool).flatten()

    # Seleciona apenas os pontos válidos
    preds_valid = predictions[mask]
    acts_valid  = actual[mask]

    if len(acts_valid) == 0:
        print(f"{model_name}: NÃO HÁ PONTOS REAIS PARA AVALIAÇÃO NESTA JANELA!")
        return np.nan, np.nan, np.nan, model_name

    errors = preds_valid - acts_valid
    mse    = np.square(errors).mean()
    rmse   = np.sqrt(mse)
    nrmse  = rmse / ((acts_valid.max() - acts_valid.min()) + 1e-12)
    mae    = np.abs(errors).mean()

    print(model_name + ' (missing-aware):')
    print(f'MAE:  {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NRMSE:{nrmse*100:.4f}%\n')

    return rmse, mae, nrmse, model_name

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    nrmse = rmse/ ((np.max(actual))-(np.min(actual)))
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Normalized Root Mean Square Error: {:.4f}%'.format(nrmse*100))
    print('')

    return rmse, mae, nrmse, model_name


#### Utils

In [ ]:
def bits_para_megabits(df, col_vaz):
    df[col_vaz] = df[col_vaz]/1000000
    df[col_vaz] = df[col_vaz].replace(-1, df[col_vaz].mean())
    df[col_vaz] = df[col_vaz].fillna(df[col_vaz].mean())

    return df

def linear_interpolation(df, limit_direction='both', method='linear'):
    df_imputed = df.interpolate(method=method, limit_direction=limit_direction)

    df['Throughput'] = df['Throughput'].fillna(df_imputed['Throughput'])

    return df


#### Plots and visualizations

In [ ]:
                
def visualizacao_series(df, col_vazao, titulo):
    df[col_vazao].plot(figsize=(18,6))
    plt.title(titulo)
    plt.ylabel('Vazao (Mbits/s)')
    plt.legend() 
    plt.show()

#plotar os graficos da media dos treinamentos por epocas: validação cruzada
def plot_loss_cv(mean_history, model_name, link):
    epochs = range(1, len(mean_history['loss']) + 1)
    plt.plot(epochs, mean_history['loss'], label='Train Loss')
    plt.plot(epochs, mean_history['val_loss'], label='Validation Loss')
    plt.title('Mean Training and Validation Loss for '+' '+link + ' '+ model_name)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_rmse_cv(mean_history):
    epochs = range(1, len(mean_history['root_mean_squared_error']) + 1)
    plt.plot(epochs, mean_history['root_mean_squared_error'], label='Train RMSE')
    plt.plot(epochs, mean_history['val_root_mean_squared_error'], label='Validation RMSE')
    plt.title('Mean Training and Validation RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend()
    plt.show()

########################################### plote dos graficos de treinamento ###################################################################################
 #Plot train loss and validation loss
def plot_loss(history, model_name, link):
     plt.figure(figsize = (15, 6), dpi=100)
     plt.plot(history.history['loss'])
     plt.plot(history.history['val_loss'])
     plt.title('Model Train vs Validation Loss for '+' '+link + ' '+ model_name)
     plt.ylabel('Loss')
     plt.xlabel('Epoch')
     plt.legend(['Train loss', 'Validation loss'], loc='upper right')
def plot_rmse(history, model_name, link):
     plt.figure(figsize = (15, 6), dpi=100)
     plt.plot(history.history['rmse'])
     plt.plot(history.history['val_rmse'])
     plt.title('Model Train vs RMSE for '+' '+link + ' '+ model_name)
     plt.ylabel('rmse')
     plt.xlabel('Epoch')
     plt.legend(['Train rmse', 'Validation loss'], loc='upper right')
################################################################################################################################################################
 

def plot_future(predictionLSTM, y_test, link):
    plt.figure(figsize=(15, 6), dpi=100)
    range_future = len(y_test)
    plt.plot(np.arange(range_future), np.array(y_test), label='Test data')
    plt.plot(np.arange(range_future), np.array(predictionLSTM), label='LSTM')
    # dict_to_dataframe_prediction = {
    #     # "range_future": np.arange(range_future),
    #     f"prediction{model_name}": np.array(prediction.squeeze())
    # }
    
    plt.title('Test data vs prediction for '+ link)
    plt.legend(loc='upper left')
    plt.xlabel('Time')
    plt.ylabel('Mbis/s')
    save_path = os.path.join('..', '..', 'results', 'bi-lstm', 'plots', link + '.png')
    save_path = os.path.normpath(save_path)  

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    try:
        plt.savefig(save_path)
        print(f"A figura foi salva com sucesso em: {save_path}")
    except Exception as e:
        print(f"Erro ao salvar a figura: {e}")
    plt.show()

    # #Tenta salvar a fig
    # save_path = os.path.join('..', '..', 'graficos', 'predicoes', 'round_2', 'graficos', link + '.png')

    # #save_path = '../../graficos/predicoes/round_2/graficos/' + link + '.png'
    # try:
    #     plt.savefig(save_path)
    #     print(f"A figura foi salva com sucesso em: {save_path}")
    # except Exception as e:
    #     print(f"Erro ao salvar a figura: {e}")

    # plt.show()


# Plot test data vs prediction
# def plot_future(predictionGRU, predictionLSTM, y_test, link):
#     plt.figure(figsize=(15, 6), dpi=100)
#     range_future = len(y_test)
#     plt.plot(np.arange(range_future), np.array(y_test), label='Test data')
#     plt.plot(np.arange(range_future), np.array(predictionGRU), label='GRU')
#     plt.plot(np.arange(range_future), np.array(predictionLSTM), label='LSTM')
#     # dict_to_dataframe_prediction = {
#     #     # "range_future": np.arange(range_future),
#     #     f"prediction{model_name}": np.array(prediction.squeeze())
#     # }
    
#     plt.title('Test data vs prediction for '+ link)
#     plt.legend(loc='upper left')
#     plt.xlabel('Time')
#     plt.ylabel('Mbis/s')

#     #Tenta salvar a fig
#     save_path = '../../graficos/predicoes/round_2/graficos/' + link + '.png'
#     try:
#         plt.savefig(save_path)
#         print(f"A figura foi salva com sucesso em: {save_path}")
#     except Exception as e:
#         print(f"Erro ao salvar a figura: {e}")

#     plt.show()
    

#### Data manipulation

##### Paths

In [ ]:
# Paths
TRAINING_OUTPUT = os.path.join('training_output2.txt')
THROUGHPUT_DATASETS = os.path.join('..', '..', 'datasets', 'test-recursive-lstm-test')
MODEL = os.path.join("..", "..", 'modelo_salvo')
METRICS = os.path.join('..', '..', 'results', 'bi-lstm', 'evaluation_rmse_mae_2.json')

In [ ]:
#função para salvar o modelo
def save_model(model, directory, substring_desejada, modelo):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{substring_desejada +modelo} - final_model.keras')
    model.save(file_path)
    print(f"Modelo salvo como '{file_path}'")

### Main
#### Model training and predcition 

#### Funtional with part of new solution

In [ ]:
def walk_forward_validation_hybrid(model, scaler, train_data, test_data, mask_test, look_back, window_size):
    """
    Executa walk-forward validation com imputação híbrida preservando estado temporal
    
    Parâmetros:
    model -- Modelo LSTM pré-treinado
    scaler -- Scaler usado na normalização
    train_data -- Dados de treino normalizados
    test_data -- Dados de teste normalizados
    mask_test -- Máscara de dados faltantes no teste
    look_back -- Tamanho da janela histórica
    window_size -- Tamanho da janela de predição
    
    Retorna:
    predictions -- Lista de previsões em escala original
    """
    # Estado inicial: últimos pontos do treino
    state = train_data[-look_back:].reshape(1, look_back, 1)
    predictions = []
    
    for i in range(0, len(test_data), window_size):
        # Seleciona janela atual
        end_idx = min(i + window_size, len(test_data))
        window_data = test_data[i:end_idx]
        window_mask = mask_test[i:end_idx]
        
        window_preds = []
        current_state = state
        
        # Predição passo-a-passo dentro da janela
        for j in range(len(window_data)):
            # Faz predição com estado atual
            pred_scaled = model.predict(current_state, verbose=0)
            pred_orig = scaler.inverse_transform(pred_scaled)[0,0]
            window_preds.append(pred_orig)
            
            # Atualiza estado com dado real ou predição
            if window_mask[j]:
                new_point = window_data[j]
            else:
                new_point = pred_scaled[0,0]
            
            # Atualiza estado: remove ponto mais antigo, adiciona novo
            current_state = np.roll(current_state, -1, axis=1)
            current_state[0, -1, 0] = new_point
        
        predictions.extend(window_preds)
        state = current_state  # Mantém estado para próxima janela
    
    return predictions


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping

# Certifique-se de importar funções auxiliares usadas no script
# from utils import bits_para_megabits, linear_interpolation, create_dataset, grid_search_cv
# from utils import create_lstm, walk_forward_validation_hybrid, validate_missing_data_prediction
# from utils import save_model, calculate_mean_history

# Variáveis globais/configuradas
LOOK_BACK = 3
WINDOW_SIZE = 28  # Tamanho da janela de predição
# TRAINING_OUTPUT = 'training_output.txt'  # Ajuste conforme necessário
# THROUGHPUT_DATASETS = './datasets'  # Caminho correto
# MODEL = './saved_models'  # Caminho correto para salvar os modelos

# Funções de plot

def plot_loss(history, title):
    plt.figure(figsize=(10, 5))
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def plot_predictions(y_true, y_pred, title):
    plt.figure(figsize=(12, 6))
    plt.plot(y_true, label='True Values')
    plt.plot(y_pred, label='Predictions')
    plt.title(title)
    plt.xlabel('Time Steps')
    plt.ylabel('Throughput (Mbps)')
    plt.legend()
    plt.show()

# Redirecionar saída padrão
orig_stdout = sys.stdout

with open(TRAINING_OUTPUT, 'w', encoding='utf-8') as f:
    sys.stdout = f

    diretorio_raiz = THROUGHPUT_DATASETS
    evaluation = {}

    for pasta_raiz, subpastas, arquivos in os.walk(diretorio_raiz):
        for arquivo in arquivos:
            if arquivo.endswith('.csv'):
                caminho_arquivo = os.path.join(pasta_raiz, arquivo)
                try:
                    partes = caminho_arquivo.split(os.sep)
                    if len(partes) >= 6:
                        substring_desejada = partes[4] + ' - ' + partes[5]
                    else:
                        substring_desejada = arquivo.replace('.csv', '')

                    # Carga dos dados
                    df = pd.read_csv(caminho_arquivo, index_col='Timestamp')

                    # Remover colunas desnecessárias, se houver
                    if '0' in df.columns:
                        df.drop('0', axis=1, inplace=True)

                    # Pré-processamento
                    bits_para_megabits(df, 'Throughput')

                    # Criar máscara ANTES de qualquer manipulação
                    mask_total = ~(df['Throughput'].isna() | (df['Throughput'] == -1))

                    # Split treino-teste
                    train_size = int(len(df.index) * 0.8)
                    train_data = df.iloc[:train_size].copy()
                    test_data = df.iloc[train_size:].copy()

                    # Interpolação APENAS no treino
                    train_data = linear_interpolation(train_data)

                    # Normalização
                    scaler = MinMaxScaler().fit(train_data[['Throughput']])
                    train_scaled = scaler.transform(train_data[['Throughput']])
                    test_scaled = scaler.transform(test_data[['Throughput']])

                    mask_test = mask_total.iloc[train_size:].values

                    # Treinamento do modelo
                    X_train, y_train = create_dataset(train_scaled, LOOK_BACK)

                    best_params = grid_search_cv(
                        create_lstm, 64, X_train, [1e-3],
                        y_train, [100,300,500], [32, 64, 128], [10], 'LSTM'
                    )

                    # Treinar modelo final
                    model = create_lstm(64, X_train, best_params['learning_rate'])
                    history = model.fit(
                        X_train, y_train,
                        epochs=best_params['epochs'],
                        batch_size=best_params['batch_size'],
                        validation_split=0.2,
                        callbacks=[EarlyStopping(
                            monitor='val_loss',
                            patience=best_params['patience'],
                            restore_best_weights=True
                        )]
                    )

                    # Predição walk-forward
                    predictions = walk_forward_validation_hybrid(
                        model, scaler, train_scaled, test_scaled,
                        mask_test, LOOK_BACK, WINDOW_SIZE
                    )

                    # Avaliação
                    min_len = min(len(predictions), len(test_scaled) - LOOK_BACK)
                    y_test_valid = test_scaled[LOOK_BACK:LOOK_BACK + min_len]
                    mask_eval = mask_test[LOOK_BACK:LOOK_BACK + min_len]

                    y_test_orig = scaler.inverse_transform(y_test_valid)
                    predictions_arr = np.array(predictions[:min_len]).reshape(-1, 1)

                    lstm_eval = validate_missing_data_prediction(
                        predictions_arr, y_test_orig, mask_eval, 'LSTM'
                    )

                    evaluation[f"{substring_desejada}, {lstm_eval[3]}"] = lstm_eval[:3]

                    print(f"RMSE para {substring_desejada}: {lstm_eval[0]:.4f}")
                    print(f"MAE para {substring_desejada}: {lstm_eval[1]:.4f}")
                    print(f"NRMSE para {substring_desejada}: {lstm_eval[3]:.4f}")

                    # Salvar e plotar
                    save_model(model, MODEL, substring_desejada, 'LSTM')
                    plot_loss(history.history, f'LSTM Loss - {substring_desejada}')
                    plot_predictions(y_test_orig, predictions_arr, f'LSTM Predictions - {substring_desejada}')
                    
                    import json  # Certifique-se de importar o json se for usá-lo

                    # Diretório para salvar predições
                    PREDICTIONS = './saved_predictions'
                    os.makedirs(PREDICTIONS, exist_ok=True)

                    # Salvar predições e valores reais em CSV
                    try:
                        pred_df = pd.DataFrame({
                            'True_Throughput': y_test_orig.flatten(),
                            'Predicted_Throughput': predictions_arr.flatten()
                        })

                        nome_base = substring_desejada.replace(' ', '_').replace('/', '_')
                        pred_path = os.path.join(PREDICTIONS, f'{nome_base}_predictions.csv')
                        pred_df.to_csv(pred_path, index=False)

                    except Exception as e:
                        print(f"Não foi possível salvar predições para {substring_desejada}: {str(e)}")

                    novo_dicionario = {}
                    
                    # Itere pelo dicionário original
                    for chave, valores in lstm_eval.items():
                        valor1, valor2, valor3 = valores  # Desempacote os valores da tupla
                        novo_dicionario[chave] = {'RMSE': valor1, 'MAE': valor2, 'NRMSE': valor3}

                    try:
                        with open(METRICS, 'w', encoding='utf-8') as f:
                            json.dump(novo_dicionario, f, indent=4)

                    except Exception as e:
                        print(f"Não foi possível salvar resultados em {METRICS}: {str(e)}")
                except Exception as e:
                    print(f"Erro no arquivo {arquivo}: {str(e)}")

    # Restaurar stdout
    sys.stdout = orig_stdout


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import RootMeanSquaredError
import tensorflow.keras as keras
import random

# ===================================================================================
# FUNÇÕES PARA SIMULAÇÃO DE FALHAS E CRIAÇÃO DE FEATURES AUXILIARES
# ===================================================================================

def identify_low_missing_regions(data, mask, min_consecutive=10, max_missing_ratio=0.1):
    """
    Identifica regiões com poucas falhas para simulação controlada
    
    Parâmetros:
    data -- dados originais
    mask -- máscara de dados válidos (True = válido, False = faltante)
    min_consecutive -- mínimo de pontos consecutivos válidos
    max_missing_ratio -- máximo ratio de dados faltantes permitido na região
    
    Retorna:
    regions -- lista de (start_idx, end_idx) das regiões identificadas
    """
    regions = []
    window_size = max(50, min_consecutive * 2)  # Janela adaptativa
    
    for i in range(0, len(data) - window_size, window_size // 2):
        window_mask = mask[i:i + window_size]
        missing_ratio = 1 - np.mean(window_mask)
        
        if missing_ratio <= max_missing_ratio:
            # Verifica se há sequências consecutivas suficientes
            consecutive_count = 0
            max_consecutive = 0
            
            for valid in window_mask:
                if valid:
                    consecutive_count += 1
                    max_consecutive = max(max_consecutive, consecutive_count)
                else:
                    consecutive_count = 0
            
            if max_consecutive >= min_consecutive:
                regions.append((i, i + window_size))
    
    return regions

def simulate_missing_data(data, mask, missing_ratio=0.2, seed=None):
    """
    Simula dados faltantes artificialmente em regiões com poucos gaps
    
    Parâmetros:
    data -- dados originais
    mask -- máscara original de dados válidos
    missing_ratio -- percentual de dados a serem removidos artificialmente
    seed -- seed para reproducibilidade
    
    Retorna:
    artificial_mask -- nova máscara com falhas artificiais
    artificial_data -- dados com falhas artificiais
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Identifica regiões com poucas falhas
    regions = identify_low_missing_regions(data, mask)
    
    if not regions:
        print("Aviso: Nenhuma região com poucos gaps encontrada")
        return mask.copy(), data.copy()
    
    artificial_mask = mask.copy()
    artificial_data = data.copy()
    
    # Para cada região, remove dados artificialmente
    for start_idx, end_idx in regions:
        region_mask = mask[start_idx:end_idx]
        valid_indices = np.where(region_mask)[0]
        
        if len(valid_indices) > 0:
            # Remove uma porcentagem dos dados válidos
            n_to_remove = int(len(valid_indices) * missing_ratio)
            indices_to_remove = np.random.choice(valid_indices, n_to_remove, replace=False)
            
            for idx in indices_to_remove:
                artificial_mask[start_idx + idx] = False
                artificial_data[start_idx + idx] = np.nan
    
    return artificial_mask, artificial_data

def create_auxiliary_features(data, mask):
    """
    Cria features auxiliares: máscara e tempo desde última observação válida
    
    Parâmetros:
    data -- série temporal
    mask -- máscara de dados válidos
    
    Retorna:
    mask_feature -- feature binária (1=real, 0=imputado)
    delta_t_feature -- tempo desde última observação real
    """
    mask_feature = mask.astype(int)
    delta_t_feature = np.zeros_like(mask, dtype=float)
    
    last_valid_idx = -1
    for i in range(len(mask)):
        if mask[i]:
            last_valid_idx = i
            delta_t_feature[i] = 0
        else:
            if last_valid_idx >= 0:
                delta_t_feature[i] = i - last_valid_idx
            else:
                delta_t_feature[i] = i + 1  # Desde o início
    
    # Normaliza delta_t
    max_delta = np.max(delta_t_feature)
    if max_delta > 0:
        delta_t_feature = delta_t_feature / max_delta
    
    return mask_feature, delta_t_feature

def create_dataset_with_auxiliary_features(X, mask_feature, delta_t_feature, look_back=3):
    """
    Cria dataset com features auxiliares para LSTM
    
    Parâmetros:
    X -- dados principais
    mask_feature -- feature de máscara
    delta_t_feature -- feature de tempo
    look_back -- janela histórica
    
    Retorna:
    Xs -- entradas com 3 features: [valor, mask, delta_t]
    ys -- saídas
    mask_ys -- máscara das saídas
    """
    Xs, ys, mask_ys = [], [], []
    
    for i in range(len(X) - look_back):
        # Features principais
        v_main = X[i:i+look_back].reshape(-1, 1)
        v_mask = mask_feature[i:i+look_back].reshape(-1, 1)
        v_delta = delta_t_feature[i:i+look_back].reshape(-1, 1)
        
        # Combina features
        v = np.concatenate([v_main, v_mask, v_delta], axis=1)
        
        Xs.append(v)
        ys.append(X[i+look_back])
        mask_ys.append(mask_feature[i+look_back])
        
    return np.array(Xs), np.array(ys), np.array(mask_ys)

# ===================================================================================
# MODELO LSTM MELHORADO COM FEATURES AUXILIARES
# ===================================================================================

def create_robust_lstm(units, train_shape, learning_rate, dropout_rate=0.2):
    """
    Cria modelo LSTM robusto com features auxiliares
    
    Parâmetros:
    units -- número de unidades LSTM
    train_shape -- formato dos dados de treino
    learning_rate -- taxa de aprendizado
    dropout_rate -- taxa de dropout
    
    Retorna:
    model -- modelo LSTM compilado
    """
    model = Sequential()
    
    # Primeira camada LSTM com dropout
    model.add(LSTM(units=units, return_sequences=True, 
                   input_shape=[train_shape[1], train_shape[2]]))
    model.add(Dropout(dropout_rate))
    
    # Segunda camada LSTM
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    
    # Camada de saída
    model.add(Dense(1))
    
    # Compilação com loss Huber (mais robusto a outliers)
    model.compile(
        loss=Huber(delta=0.5),
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[RootMeanSquaredError()]
    )
    
    return model

def weighted_loss_function(y_true, y_pred, sample_weight=None):
    """
    Função de perda ponderada baseada na máscara
    """
    if sample_weight is None:
        return keras.losses.huber(y_true, y_pred)
    else:
        loss = keras.losses.huber(y_true, y_pred)
        return loss * sample_weight

# ===================================================================================
# GRID SEARCH COM VALIDAÇÃO CRUZADA MELHORADA
# ===================================================================================

def grid_search_cv_robust(modelo, units, X_train, learning_rates, y_train, mask_train,
                         epochs_list, batch_sizes, patiences, model_name):
    """
    Grid search com validação cruzada considerando dados faltantes
    """
    best_loss = float('inf')
    best_params = {}
    
    for lr in learning_rates:
        for epochs in epochs_list:
            for batch_size in batch_sizes:
                for patience in patiences:
                    model = modelo(units, X_train, lr)
                    histories = fit_model_with_cross_validation_robust(
                        model, X_train, y_train, mask_train, model_name, 
                        patience, epochs, batch_size
                    )
                    mean_history = calculate_mean_history(histories)
                    val_loss = min(mean_history['val_loss'])
                    
                    print(f"Val Loss: {val_loss:.4f}, lr: {lr}, epochs: {epochs}, "
                          f"batch_size: {batch_size}, patience: {patience}")
                    
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_params = {
                            'learning_rate': lr, 
                            'epochs': epochs, 
                            'batch_size': batch_size, 
                            'patience': patience
                        }
    
    print(f'Melhores parâmetros para {model_name}: {best_params}')
    return best_params

def fit_model_with_cross_validation_robust(model, xtrain, ytrain, mask_train, 
                                          model_name, patience, epochs, batch_size):
    """
    Validação cruzada com pesos baseados na máscara
    """
    tscv = TimeSeriesSplit(n_splits=5)
    fold = 1
    histories = []
    
    for train_index, val_index in tscv.split(xtrain):
        x_train_fold, x_val_fold = xtrain[train_index], xtrain[val_index]
        y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
        mask_train_fold = mask_train[train_index]
        mask_val_fold = mask_train[val_index]
        
        # Pesos baseados na máscara (dados reais têm peso maior)
        sample_weights = np.where(mask_train_fold, 1.0, 0.3)
        val_sample_weights = np.where(mask_val_fold, 1.0, 0.3)
        
        early_stop = EarlyStopping(
            monitor='val_loss', 
            patience=patience, 
            restore_best_weights=True, 
            min_delta=1e-5
        )
        
        history = model.fit(
            x_train_fold, y_train_fold,
            epochs=epochs,
            validation_data=(x_val_fold, y_val_fold, val_sample_weights),
            batch_size=batch_size,
            sample_weight=sample_weights,
            callbacks=[early_stop],
            verbose=1
        )
        
        print(f'\nTREINAMENTO - Fold {fold} do modelo: {model_name}')
        histories.append(history)
        fold += 1
    
    return histories

def calculate_mean_history(histories):
    """
    Calcula média dos históricos de treinamento
    """
    mean_history = {
        'loss': [], 
        'root_mean_squared_error': [], 
        'val_loss': [], 
        'val_root_mean_squared_error': []
    }
    
    for fold_history in histories:
        for key in mean_history.keys():
            if key in fold_history.history:
                mean_history[key].append(fold_history.history[key])
    
    # Equaliza comprimentos
    for key, values in mean_history.items():
        if values:
            max_len = max(len(val) for val in values)
            for i in range(len(values)):
                if len(values[i]) < max_len:
                    values[i] += [values[i][-1]] * (max_len - len(values[i]))
    
    # Calcula médias
    for key, values in mean_history.items():
        if values:
            mean_history[key] = [sum(vals) / len(vals) for vals in zip(*values)]
    
    return mean_history

# ===================================================================================
# WALK-FORWARD VALIDATION MELHORADO
# ===================================================================================

def walk_forward_validation_robust(model, scaler, train_data, test_data, mask_test,
                                  mask_feature_test, delta_t_feature_test,
                                  look_back, window_size):
    """
    Walk-forward validation robusto com features auxiliares
    """
    # Preparar dados de entrada com features auxiliares
    # Estado inicial: últimos pontos do treino
    state_main = train_data[-look_back:]
    state_mask = np.ones(look_back)  # Assume que dados de treino são válidos
    state_delta = np.zeros(look_back)
    
    # Combinar features do estado inicial
    state = np.stack([state_main, state_mask, state_delta], axis=1).reshape(1, look_back, 3)
    
    predictions = []
    
    for i in range(0, len(test_data), window_size):
        end_idx = min(i + window_size, len(test_data))
        window_data = test_data[i:end_idx]
        window_mask = mask_test[i:end_idx]
        window_mask_feature = mask_feature_test[i:end_idx]
        window_delta_feature = delta_t_feature_test[i:end_idx]
        
        window_preds = []
        current_state = state
        
        for j in range(len(window_data)):
            # Predição com estado atual
            pred_scaled = model.predict(current_state, verbose=0)
            pred_orig = scaler.inverse_transform(pred_scaled.reshape(-1, 1))[0, 0]
            window_preds.append(pred_orig)
            
            # Atualizar estado
            if window_mask[j]:
                new_point_main = window_data[j]
            else:
                new_point_main = pred_scaled[0, 0]
            
            new_point_mask = window_mask_feature[j]
            new_point_delta = window_delta_feature[j]
            
            # Atualizar estado: shift e adicionar novo ponto
            current_state = np.roll(current_state, -1, axis=1)
            current_state[0, -1, 0] = new_point_main
            current_state[0, -1, 1] = new_point_mask
            current_state[0, -1, 2] = new_point_delta
        
        predictions.extend(window_preds)
        state = current_state
    
    return predictions

# ===================================================================================
# AVALIAÇÃO COM MÚLTIPLAS SIMULAÇÕES
# ===================================================================================

def evaluate_robustness_multiple_simulations(data, mask, n_simulations=10, 
                                           missing_ratios=[0.1, 0.2, 0.3]):
    """
    Avalia robustez com múltiplas simulações de dados faltantes
    """
    results = {}
    
    for missing_ratio in missing_ratios:
        rmse_list = []
        mae_list = []
        
        for sim in range(n_simulations):
            # Simula dados faltantes
            artificial_mask, artificial_data = simulate_missing_data(
                data, mask, missing_ratio, seed=sim
            )
            
            # Aqui você executaria todo o pipeline de treinamento
            # Por simplicidade, vou apenas simular métricas
            # Na implementação real, você substituiria por:
            # rmse, mae = train_and_evaluate_model(artificial_data, artificial_mask)
            
            # Simulação de métricas (substituir por código real)
            rmse = np.random.normal(0.5, 0.1)  # Exemplo
            mae = np.random.normal(0.3, 0.05)  # Exemplo
            
            rmse_list.append(rmse)
            mae_list.append(mae)
        
        results[missing_ratio] = {
            'rmse_mean': np.mean(rmse_list),
            'rmse_std': np.std(rmse_list),
            'mae_mean': np.mean(mae_list),
            'mae_std': np.std(mae_list)
        }
    
    return results

# ===================================================================================
# EXEMPLO DE USO INTEGRADO
# ===================================================================================

def processo_completo_robusto(df, substring_desejada):
    """
    Exemplo de como integrar todas as melhorias
    """
    # Pré-processamento
    mask_total = ~(df['Throughput'].isna() | (df['Throughput'] == -1))
    
    # Split treino-teste
    train_size = int(len(df.index) * 0.8)
    train_data = df.iloc[:train_size].copy()
    test_data = df.iloc[train_size:].copy()
    
    # Criar máscara para teste
    mask_test = mask_total.iloc[train_size:].values
    
    # Simular dados faltantes no treino para melhor robustez
    train_mask = mask_total.iloc[:train_size].values
    artificial_mask, artificial_data = simulate_missing_data(
        train_data['Throughput'].values, train_mask, missing_ratio=0.15
    )
    
    # Aplicar interpolação nos dados com falhas artificiais
    train_data_artificial = train_data.copy()
    train_data_artificial['Throughput'] = artificial_data
    # linear_interpolation(train_data_artificial)  # Sua função de interpolação
    
    # Criar features auxiliares
    mask_feature_train, delta_t_feature_train = create_auxiliary_features(
        train_data_artificial['Throughput'].values, artificial_mask
    )
    mask_feature_test, delta_t_feature_test = create_auxiliary_features(
        test_data['Throughput'].values, mask_test
    )
    
    # Normalização
    scaler = MinMaxScaler().fit(train_data_artificial[['Throughput']])
    train_scaled = scaler.transform(train_data_artificial[['Throughput']])
    test_scaled = scaler.transform(test_data[['Throughput']])
    
    # Criar datasets com features auxiliares
    X_train, y_train, mask_y_train = create_dataset_with_auxiliary_features(
        train_scaled.flatten(), mask_feature_train, delta_t_feature_train, look_back=3
    )
    
    # Grid search robusto
    best_params = grid_search_cv_robust(
        create_robust_lstm, 64, X_train, [1e-3, 1e-4],
        y_train, mask_y_train, [100, 200], [32, 64], [10], 'LSTM-Robust'
    )
    
    # Treinar modelo final
    model = create_robust_lstm(64, X_train.shape, best_params['learning_rate'])
    
    # Pesos para treinamento
    sample_weights = np.where(mask_y_train, 1.0, 0.3)
    
    history = model.fit(
        X_train, y_train,
        epochs=best_params['epochs'],
        batch_size=best_params['batch_size'],
        sample_weight=sample_weights,
        validation_split=0.2,
        callbacks=[EarlyStopping(
            monitor='val_loss',
            patience=best_params['patience'],
            restore_best_weights=True
        )]
    )
    
    # Predição robusta
    predictions = walk_forward_validation_robust(
        model, scaler, train_scaled.flatten(), test_scaled.flatten(),
        mask_test, mask_feature_test, delta_t_feature_test,
        look_back=3, window_size=28
    )
    
    return model, predictions, history

# ===================================================================================
# MÉTRICAS DE AVALIAÇÃO MELHORADAS
# ===================================================================================

def validate_missing_data_prediction_robust(predictions, actual, mask, model_name):
    """
    Versão melhorada da função de validação considerando incerteza
    """
    predictions = predictions.flatten()
    actual = actual.flatten()
    mask = mask.astype(bool).flatten()
    
    # Avaliação apenas em pontos válidos
    preds_valid = predictions[mask]
    acts_valid = actual[mask]
    
    if len(acts_valid) == 0:
        print(f"{model_name}: Nenhum ponto válido para avaliação!")
        return np.nan, np.nan, np.nan, model_name
    
    # Métricas tradicionais
    errors = preds_valid - acts_valid
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    nrmse = rmse / (acts_valid.max() - acts_valid.min() + 1e-12)
    
    # Métricas adicionais de robustez
    median_error = np.median(np.abs(errors))
    q75_error = np.percentile(np.abs(errors), 75)
    
    print(f'{model_name} (robust evaluation):')
    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NRMSE: {nrmse*100:.4f}%')
    print(f'Median Error: {median_error:.4f}')
    print(f'75th Percentile Error: {q75_error:.4f}')
    print(f'Valid points: {len(acts_valid)}/{len(actual)} ({len(acts_valid)/len(actual)*100:.1f}%)')
    print('')
    
    return rmse, mae, nrmse, model_name

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import json

# Importar funções do código melhorado
# from robust_lstm_functions import *  # Todas as funções do código anterior

# Configurações
BASE_DIR = "resultados-recursive-prediction-robust"
PREDICTIONS_DIR = os.path.join(BASE_DIR, "predicoes")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
PLOTS_LOSS_DIR = os.path.join(PLOTS_DIR, "loss")
PLOTS_PREDICTIONS_DIR = os.path.join(PLOTS_DIR, "predicoes")
TRAINING_OUTPUT = os.path.join(BASE_DIR, 'training_output_robust.txt')
METRICS_PATH = os.path.join(BASE_DIR, 'metrics_robust.json')

# Criar diretórios
for dir_path in [BASE_DIR, PREDICTIONS_DIR, PLOTS_DIR, PLOTS_LOSS_DIR, PLOTS_PREDICTIONS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

LOOK_BACK = 3
WINDOW_SIZE = 28
TRAINING_OUTPUT = os.path.join('training_output2.txt')
THROUGHPUT_DATASETS = os.path.join('..', '..', 'datasets', 'test-recursive-lstm-test')
MODEL_DIR = './saved_models_robust'

# Função principal melhorada
def processo_arquivo_robusto(caminho_arquivo, arquivo):
    """
    Processa um arquivo CSV com o pipeline robusto completo
    """
    try:
        # Extrair nome do arquivo
        partes = caminho_arquivo.split(os.sep)
        if len(partes) >= 6:
            substring_desejada = partes[4] + ' - ' + partes[5]
        else:
            substring_desejada = arquivo.replace('.csv', '')
        
        print(f"\n{'='*50}")
        print(f"Processando: {substring_desejada}")
        print(f"{'='*50}")
        
        # Carregar dados
        df = pd.read_csv(caminho_arquivo, index_col='Timestamp')
        
        # Remover colunas desnecessárias
        if '0' in df.columns:
            df.drop('0', axis=1, inplace=True)
        
        # Pré-processamento
        # bits_para_megabits(df, 'Throughput')  # Sua função
        
        # Criar máscara original
        mask_total = ~(df['Throughput'].isna() | (df['Throughput'] == -1))
        
        # Verificar se há dados suficientes
        if np.sum(mask_total) < 100:
            print(f"Dados insuficientes para {substring_desejada}")
            return None
        
        # Split treino-teste
        train_size = int(len(df.index) * 0.8)
        train_data = df.iloc[:train_size].copy()
        test_data = df.iloc[train_size:].copy()
        
        # Máscaras
        mask_train = mask_total.iloc[:train_size].values
        mask_test = mask_total.iloc[train_size:].values
        
        print(f"Dados de treino: {len(train_data)} pontos, {np.sum(mask_train)} válidos ({np.sum(mask_train)/len(mask_train)*100:.1f}%)")
        print(f"Dados de teste: {len(test_data)} pontos, {np.sum(mask_test)} válidos ({np.sum(mask_test)/len(mask_test)*100:.1f}%)")
        
        # ===================================================================================
        # ESTRATÉGIA 1: SIMULAÇÃO DE FALHAS ARTIFICIAIS
        # ===================================================================================
        
        print("\n1. Simulando falhas artificiais no treino...")
        
        # Identificar regiões com poucos gaps
        regions = identify_low_missing_regions(train_data['Throughput'].values, mask_train)
        print(f"Regiões com poucos gaps identificadas: {len(regions)}")
        
        # Simular falhas artificiais (15% dos dados válidos)
        artificial_mask, artificial_data = simulate_missing_data(
            train_data['Throughput'].values, mask_train, 
            missing_ratio=0.15, seed=42
        )
        
        original_valid = np.sum(mask_train)
        artificial_valid = np.sum(artificial_mask)
        print(f"Dados válidos: {original_valid} -> {artificial_valid} (removidos: {original_valid - artificial_valid})")
        
        # Aplicar interpolação nos dados com falhas artificiais
        train_data_artificial = train_data.copy()
        train_data_artificial['Throughput'] = artificial_data
        
        # Interpolação linear simples (substitua pela sua função)
        train_data_artificial['Throughput'] = train_data_artificial['Throughput'].interpolate(method='linear')
        
        # ===================================================================================
        # ESTRATÉGIA 2: FEATURES AUXILIARES
        # ===================================================================================
        
        print("\n2. Criando features auxiliares...")
        
        # Criar features auxiliares para treino
        mask_feature_train, delta_t_feature_train = create_auxiliary_features(
            train_data_artificial['Throughput'].values, artificial_mask
        )
        
        # Criar features auxiliares para teste
        test_data_interpolated = test_data.copy()
        test_data_interpolated['Throughput'] = test_data_interpolated['Throughput'].interpolate(method='linear')
        
        mask_feature_test, delta_t_feature_test = create_auxiliary_features(
            test_data_interpolated['Throughput'].values, mask_test
        )
        
        print(f"Features auxiliares criadas: mask + delta_t")
        
        # ===================================================================================
        # ESTRATÉGIA 3: NORMALIZAÇÃO E DATASET COM FEATURES AUXILIARES
        # ===================================================================================
        
        print("\n3. Preparando dados com features auxiliares...")
        
        # Normalização
        scaler = MinMaxScaler().fit(train_data_artificial[['Throughput']])
        train_scaled = scaler.transform(train_data_artificial[['Throughput']])
        test_scaled = scaler.transform(test_data_interpolated[['Throughput']])
        
        # Criar datasets com features auxiliares
        X_train, y_train, mask_y_train = create_dataset_with_auxiliary_features(
            train_scaled.flatten(), mask_feature_train, delta_t_feature_train, look_back=LOOK_BACK
        )
        
        print(f"Dataset de treino: {X_train.shape} (3 features: valor, mask, delta_t)")
        
        # ===================================================================================
        # ESTRATÉGIA 4: GRID SEARCH ROBUSTO
        # ===================================================================================
        
        print("\n4. Executando grid search robusto...")
        
        best_params = grid_search_cv_robust(
            create_robust_lstm, 64, X_train, [1e-3, 1e-4],
            y_train, mask_y_train, [100, 200, 300], [32, 64], [10], 'LSTM-Robust'
        )
        
        # ===================================================================================
        # ESTRATÉGIA 5: TREINAMENTO COM LOSS PONDERADA
        # ===================================================================================
        
        print("\n5. Treinando modelo com loss ponderada...")
        
        # Criar modelo final
        model = create_robust_lstm(64, X_train.shape, 5)
        
                # Pesos para treinamento (dados reais têm peso maior)
        sample_weights = np.where(mask_y_train, 1.0, 0.3)
        
        # Early stopping
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=best_params['patience'],
            restore_best_weights=True,
            min_delta=1e-5
        )
        
        # Treinar modelo final
        history = model.fit(
            X_train, y_train,
            epochs=best_params['epochs'],
            batch_size=best_params['batch_size'],
            validation_split=0.2,
            sample_weight=sample_weights,
            callbacks=[early_stop],
            verbose=1
        )
        
        # ===================================================================================
        # ESTRATÉGIA 6: PREDIÇÃO WALK-FORWARD ROBUSTA
        # ===================================================================================
        
        print("\n6. Realizando predição walk-forward robusta...")
        
        # Preparar dados de teste
        test_scaled_flat = test_scaled.flatten()
        
        predictions = walk_forward_validation_robust(
            model, scaler, 
            train_scaled.flatten(),  # Últimos pontos do treino
            test_scaled_flat,
            mask_test,
            mask_feature_test,
            delta_t_feature_test,
            look_back=LOOK_BACK,
            window_size=WINDOW_SIZE
        )
        
        # Converter predições para formato original
        predictions = np.array(predictions).reshape(-1, 1)
        predictions_orig = scaler.inverse_transform(predictions)
        
        # ===================================================================================
        # ESTRATÉGIA 7: AVALIAÇÃO ROBUSTA E SALVAMENTO DE RESULTADOS
        # ===================================================================================
        
        print("\n7. Avaliando e salvando resultados...")
        
        # Validar predições considerando máscara
        actual_values = test_data['Throughput'].values[LOOK_BACK:]
        mask_valid = mask_test[LOOK_BACK:]
        
        rmse, mae, nrmse, _ = validate_missing_data_prediction_robust(
            predictions_orig, 
            actual_values.reshape(-1, 1), 
            mask_valid,
            substring_desejada
        )
        
        # Salvar métricas
        metrics = {
            'RMSE': rmse,
            'MAE': mae,
            'NRMSE': nrmse,
            'Valid_Points': f"{np.sum(mask_valid)}/{len(mask_valid)}",
            'Valid_Percentage': f"{np.sum(mask_valid)/len(mask_valid)*100:.1f}%"
        }
        
        # Salvar predições
        pred_df = pd.DataFrame({
            'Timestamp': test_data.index[LOOK_BACK:],
            'Actual': actual_values,
            'Predicted': predictions_orig.flatten(),
            'Valid': mask_valid
        })
        pred_file = os.path.join(PREDICTIONS_DIR, f"{substring_desejada}_predictions.csv")
        pred_df.to_csv(pred_file, index=False)
        
        # Salvar modelo
        model_file = os.path.join(MODEL_DIR, f"{substring_desejada}_model.keras")
        model.save(model_file)
        
        # Salvar histórico de treinamento
        history_file = os.path.join(BASE_DIR, f"{substring_desejada}_history.json")
        with open(history_file, 'w') as f:
            json.dump(history.history, f)
            
        # Plotar perda de treinamento
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Training History: {substring_desejada}')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True)
        loss_file = os.path.join(PLOTS_LOSS_DIR, f"{substring_desejada}_loss.png")
        plt.savefig(loss_file)
        plt.close()
        
        # Plotar predições vs reais
        plt.figure(figsize=(12, 6))
        valid_indices = np.where(mask_valid)[0]
        plt.plot(pred_df['Timestamp'], pred_df['Actual'], 'b-', label='Actual')
        plt.plot(pred_df['Timestamp'], pred_df['Predicted'], 'r-', alpha=0.7, label='Predicted')
        plt.scatter(
            pred_df.iloc[valid_indices]['Timestamp'],
            pred_df.iloc[valid_indices]['Actual'],
            c='green', s=15, label='Valid Points'
        )
        plt.title(f'Prediction vs Actual: {substring_desejada}')
        plt.xlabel('Timestamp')
        plt.ylabel('Throughput')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        pred_file = os.path.join(PLOTS_PREDICTIONS_DIR, f"{substring_desejada}_prediction.png")
        plt.savefig(pred_file)
        plt.close()
        
        # Registrar métricas no arquivo de saída
        with open(TRAINING_OUTPUT, 'a') as f:
            f.write(f"\n\n{'='*50}\n")
            f.write(f"Resultados para: {substring_desejada}\n")
            f.write(f"RMSE: {rmse:.4f}\n")
            f.write(f"MAE: {mae:.4f}\n")
            f.write(f"NRMSE: {nrmse*100:.2f}%\n")
            f.write(f"Valid Points: {np.sum(mask_valid)}/{len(mask_valid)} ({np.sum(mask_valid)/len(mask_valid)*100:.1f}%)\n")
        
        # Atualizar métricas globais
        global_metrics = {}
        if os.path.exists(METRICS_PATH):
            with open(METRICS_PATH, 'r') as f:
                global_metrics = json.load(f)
                
        global_metrics[substring_desejada] = metrics
        
        with open(METRICS_PATH, 'w') as f:
            json.dump(global_metrics, f, indent=4)
            
        print(f"Processamento completo para {substring_desejada}")
        return metrics
        
    except Exception as e:
        print(f"Erro no processamento de {arquivo}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Função principal para executar o pipeline
def main():
    arquivos_processados = 0
    
    # Inicializar arquivo de saída
    with open(TRAINING_OUTPUT, 'w') as f:
        f.write("=== INÍCIO DO TREINAMENTO ROBUSTO ===\n")
    
    # Coletar todos os arquivos CSV
    arquivos = []
    for root, dirs, files in os.walk(THROUGHPUT_DATASETS):
        for file in files:
            if file.endswith('.csv'):
                arquivos.append((os.path.join(root, file), file))
    
    # Processar cada arquivo
    for caminho, arquivo in arquivos:
        resultado = processo_arquivo_robusto(caminho, arquivo)
        if resultado:
            arquivos_processados += 1
            
    print(f"\n{'='*50}")
    print(f"Processamento completo!")
    print(f"Arquivos processados: {arquivos_processados}/{len(arquivos)}")
    print(f"Resultados salvos em: {BASE_DIR}")
    print(f"{'='*50}")

if __name__ == "__main__":
    main()

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import RootMeanSquaredError
import tensorflow.keras as keras
import json
import random

# ===================================================================================
# FUNÇÕES PARA SIMULAÇÃO DE FALHAS E CRIAÇÃO DE FEATURES AUXILIARES
# ===================================================================================

def identify_low_missing_regions(data, mask, min_consecutive=10, max_missing_ratio=0.1):
    """
    Identifica regiões com poucas falhas para simulação controlada
    """
    regions = []
    window_size = max(50, min_consecutive * 2)
    
    for i in range(0, len(data) - window_size, window_size // 2):
        window_mask = mask[i:i + window_size]
        missing_ratio = 1 - np.mean(window_mask)
        
        if missing_ratio <= max_missing_ratio:
            consecutive_count = 0
            max_consecutive = 0
            
            for valid in window_mask:
                if valid:
                    consecutive_count += 1
                    max_consecutive = max(max_consecutive, consecutive_count)
                else:
                    consecutive_count = 0
            
            if max_consecutive >= min_consecutive:
                regions.append((i, i + window_size))
    
    return regions

def simulate_missing_data(data, mask, missing_ratio=0.2, seed=None):
    """
    Simula dados faltantes artificialmente em regiões com poucos gaps
    """
    if seed is not None:
        np.random.seed(seed)
    
    regions = identify_low_missing_regions(data, mask)
    
    if not regions:
        print("Aviso: Nenhuma região com poucos gaps encontrada")
        return mask.copy(), data.copy()
    
    artificial_mask = mask.copy()
    artificial_data = data.copy()
    
    for start_idx, end_idx in regions:
        region_mask = mask[start_idx:end_idx]
        valid_indices = np.where(region_mask)[0]
        
        if len(valid_indices) > 0:
            n_to_remove = int(len(valid_indices) * missing_ratio)
            indices_to_remove = np.random.choice(valid_indices, n_to_remove, replace=False)
            
            for idx in indices_to_remove:
                artificial_mask[start_idx + idx] = False
                artificial_data[start_idx + idx] = np.nan
    
    return artificial_mask, artificial_data

def create_auxiliary_features(data, mask):
    """
    Cria features auxiliares: máscara e tempo desde última observação válida
    """
    mask_feature = mask.astype(int)
    delta_t_feature = np.zeros_like(mask, dtype=float)
    
    last_valid_idx = -1
    for i in range(len(mask)):
        if mask[i]:
            last_valid_idx = i
            delta_t_feature[i] = 0
        else:
            if last_valid_idx >= 0:
                delta_t_feature[i] = i - last_valid_idx
            else:
                delta_t_feature[i] = i + 1
    
    max_delta = np.max(delta_t_feature)
    if max_delta > 0:
        delta_t_feature = delta_t_feature / max_delta
    
    return mask_feature, delta_t_feature

def create_dataset_with_auxiliary_features(X, mask_feature, delta_t_feature, look_back=3):
    """
    Cria dataset com features auxiliares para LSTM
    """
    Xs, ys, mask_ys = [], [], []
    
    for i in range(len(X) - look_back):
        v_main = X[i:i+look_back].reshape(-1, 1)
        v_mask = mask_feature[i:i+look_back].reshape(-1, 1)
        v_delta = delta_t_feature[i:i+look_back].reshape(-1, 1)
        
        v = np.concatenate([v_main, v_mask, v_delta], axis=1)
        
        Xs.append(v)
        ys.append(X[i+look_back])
        mask_ys.append(mask_feature[i+look_back])
        
    return np.array(Xs), np.array(ys), np.array(mask_ys)

# ===================================================================================
# MODELO LSTM MELHORADO COM FEATURES AUXILIARES
# ===================================================================================

def create_robust_lstm(units, input_shape, learning_rate, dropout_rate=0.2):
    """
    Cria modelo LSTM robusto com features auxiliares
    
    CORREÇÃO: Agora recebe input_shape como tupla (timesteps, features)
    """
    model = Sequential()
    
    # Primeira camada LSTM com dropout
    model.add(LSTM(units=units, return_sequences=True, 
                   input_shape=input_shape))  # input_shape deve ser (timesteps, features)
    model.add(Dropout(dropout_rate))
    
    # Segunda camada LSTM
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    
    # Camada de saída
    model.add(Dense(1))
    
    # Compilação com loss Huber
    model.compile(
        loss=Huber(delta=0.5),
        optimizer=Adam(learning_rate=5),
        metrics=[RootMeanSquaredError()]
    )
    
    return model

# ===================================================================================
# GRID SEARCH COM VALIDAÇÃO CRUZADA MELHORADA
# ===================================================================================

def grid_search_cv_robust(modelo, units, X_train, learning_rates, y_train, mask_train,
                         epochs_list, batch_sizes, patiences, model_name):
    """
    Grid search com validação cruzada considerando dados faltantes
    
    CORREÇÃO: Passa input_shape corretamente
    """
    best_loss = float('inf')
    best_params = {}
    
    # Extrair shape dos dados de treino
    input_shape = (X_train.shape[1], X_train.shape[2])  # (timesteps, features)
    
    for lr in learning_rates:
        for epochs in epochs_list:
            for batch_size in batch_sizes:
                for patience in patiences:
                    # CORREÇÃO: Passar input_shape ao invés de X_train
                    model = modelo(units, input_shape, lr)
                    histories = fit_model_with_cross_validation_robust(
                        model, X_train, y_train, mask_train, model_name, 
                        patience, epochs, batch_size
                    )
                    mean_history = calculate_mean_history(histories)
                    val_loss = min(mean_history['val_loss'])
                    
                    print(f"Val Loss: {val_loss:.4f}, lr: {lr}, epochs: {epochs}, "
                          f"batch_size: {batch_size}, patience: {patience}")
                    
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_params = {
                            'learning_rate': 5, 
                            'epochs': epochs, 
                            'batch_size': batch_size, 
                            'patience': patience
                        }
    
    print(f'Melhores parâmetros para {model_name}: {best_params}')
    return best_params

def fit_model_with_cross_validation_robust(model, xtrain, ytrain, mask_train, 
                                          model_name, patience, epochs, batch_size):
    """
    Validação cruzada com pesos baseados na máscara
    """
    tscv = TimeSeriesSplit(n_splits=5)
    fold = 1
    histories = []
    
    for train_index, val_index in tscv.split(xtrain):
        x_train_fold, x_val_fold = xtrain[train_index], xtrain[val_index]
        y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
        mask_train_fold = mask_train[train_index]
        mask_val_fold = mask_train[val_index]
        
        # Pesos baseados na máscara
        sample_weights = np.where(mask_train_fold, 1.0, 0.3)
        val_sample_weights = np.where(mask_val_fold, 1.0, 0.3)
        
        early_stop = EarlyStopping(
            monitor='val_loss', 
            patience=patience, 
            restore_best_weights=True, 
            min_delta=1e-5
        )
        
        history = model.fit(
            x_train_fold, y_train_fold,
            epochs=epochs,
            validation_data=(x_val_fold, y_val_fold, val_sample_weights),
            batch_size=batch_size,
            sample_weight=sample_weights,
            callbacks=[early_stop],
            verbose=1
        )
        
        print(f'\nTREINAMENTO - Fold {fold} do modelo: {model_name}')
        histories.append(history)
        fold += 1
    
    return histories

def calculate_mean_history(histories):
    """
    Calcula média dos históricos de treinamento
    """
    mean_history = {
        'loss': [], 
        'root_mean_squared_error': [], 
        'val_loss': [], 
        'val_root_mean_squared_error': []
    }
    
    for fold_history in histories:
        for key in mean_history.keys():
            if key in fold_history.history:
                mean_history[key].append(fold_history.history[key])
    
    # Equaliza comprimentos
    for key, values in mean_history.items():
        if values:
            max_len = max(len(val) for val in values)
            for i in range(len(values)):
                if len(values[i]) < max_len:
                    values[i] += [values[i][-1]] * (max_len - len(values[i]))
    
    # Calcula médias
    for key, values in mean_history.items():
        if values:
            mean_history[key] = [sum(vals) / len(vals) for vals in zip(*values)]
    
    return mean_history

# ===================================================================================
# WALK-FORWARD VALIDATION MELHORADO
# ===================================================================================

def walk_forward_validation_robust(model, scaler, train_data, test_data, mask_test,
                                  mask_feature_test, delta_t_feature_test,
                                  look_back, window_size):
    """
    Walk-forward validation robusto com features auxiliares
    """
    # Estado inicial: últimos pontos do treino
    state_main = train_data[-look_back:]
    state_mask = np.ones(look_back)
    state_delta = np.zeros(look_back)
    
    # Combinar features do estado inicial
    state = np.stack([state_main, state_mask, state_delta], axis=1).reshape(1, look_back, 3)
    
    predictions = []
    
    for i in range(0, len(test_data), window_size):
        end_idx = min(i + window_size, len(test_data))
        window_data = test_data[i:end_idx]
        window_mask = mask_test[i:end_idx]
        window_mask_feature = mask_feature_test[i:end_idx]
        window_delta_feature = delta_t_feature_test[i:end_idx]
        
        window_preds = []
        current_state = state
        
        for j in range(len(window_data)):
            # Predição com estado atual
            pred_scaled = model.predict(current_state, verbose=0)
            pred_orig = scaler.inverse_transform(pred_scaled.reshape(-1, 1))[0, 0]
            window_preds.append(pred_orig)
            
            # Atualizar estado
            if window_mask[j]:
                new_point_main = window_data[j]
            else:
                new_point_main = pred_scaled[0, 0]
            
            new_point_mask = window_mask_feature[j]
            new_point_delta = window_delta_feature[j]
            
            # Atualizar estado: shift e adicionar novo ponto
            current_state = np.roll(current_state, -1, axis=1)
            current_state[0, -1, 0] = new_point_main
            current_state[0, -1, 1] = new_point_mask
            current_state[0, -1, 2] = new_point_delta
        
        predictions.extend(window_preds)
        state = current_state
    
    return predictions

# ===================================================================================
# AVALIAÇÃO COM MÚLTIPLAS MÉTRICAS
# ===================================================================================

def validate_missing_data_prediction_robust(predictions, actual, mask, model_name):
    """
    Versão melhorada da função de validação considerando incerteza
    """
    predictions = predictions.flatten()
    actual = actual.flatten()
    mask = mask.astype(bool).flatten()
    
    # Avaliação apenas em pontos válidos
    preds_valid = predictions[mask]
    acts_valid = actual[mask]
    
    if len(acts_valid) == 0:
        print(f"{model_name}: Nenhum ponto válido para avaliação!")
        return np.nan, np.nan, np.nan, model_name
    
    # Métricas tradicionais
    errors = preds_valid - acts_valid
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    nrmse = rmse / (acts_valid.max() - acts_valid.min() + 1e-12)
    
    # Métricas adicionais de robustez
    median_error = np.median(np.abs(errors))
    q75_error = np.percentile(np.abs(errors), 75)
    
    print(f'{model_name} (robust evaluation):')
    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NRMSE: {nrmse*100:.4f}%')
    print(f'Median Error: {median_error:.4f}')
    print(f'75th Percentile Error: {q75_error:.4f}')
    print(f'Valid points: {len(acts_valid)}/{len(actual)} ({len(acts_valid)/len(actual)*100:.1f}%)')
    print('')
    
    return rmse, mae, nrmse, model_name

# ===================================================================================
# CONFIGURAÇÕES E FUNÇÃO PRINCIPAL
# ===================================================================================

# Configurações
BASE_DIR = "resultados-recursive-prediction-robust"
PREDICTIONS_DIR = os.path.join(BASE_DIR, "predicoes")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
PLOTS_LOSS_DIR = os.path.join(PLOTS_DIR, "loss")
PLOTS_PREDICTIONS_DIR = os.path.join(PLOTS_DIR, "predicoes")
TRAINING_OUTPUT = os.path.join(BASE_DIR, 'training_output_robust.txt')
METRICS_PATH = os.path.join(BASE_DIR, 'metrics_robust.json')
MODEL_DIR = os.path.join(BASE_DIR, 'saved_models_robust')

# Criar diretórios
for dir_path in [BASE_DIR, PREDICTIONS_DIR, PLOTS_DIR, PLOTS_LOSS_DIR, PLOTS_PREDICTIONS_DIR, MODEL_DIR]:
    os.makedirs(dir_path, exist_ok=True)

LOOK_BACK = 3
WINDOW_SIZE = 28
THROUGHPUT_DATASETS = os.path.join('..', '..', 'datasets', 'test-recursive-lstm-test')

def processo_arquivo_robusto(caminho_arquivo, arquivo):
    """
    Processa um arquivo CSV com o pipeline robusto completo
    """
    try:
        # Extrair nome do arquivo
        partes = caminho_arquivo.split(os.sep)
        if len(partes) >= 6:
            substring_desejada = partes[4] + ' - ' + partes[5]
        else:
            substring_desejada = arquivo.replace('.csv', '')
        
        print(f"\n{'='*50}")
        print(f"Processando: {substring_desejada}")
        print(f"{'='*50}")
        
        # Carregar dados
        df = pd.read_csv(caminho_arquivo, index_col='Timestamp')
        
        # Remover colunas desnecessárias
        if '0' in df.columns:
            df.drop('0', axis=1, inplace=True)
        
        # Criar máscara original
        mask_total = ~(df['Throughput'].isna() | (df['Throughput'] == -1))
        
        # Verificar se há dados suficientes
        if np.sum(mask_total) < 100:
            print(f"Dados insuficientes para {substring_desejada}")
            return None
        
        # Split treino-teste
        train_size = int(len(df.index) * 0.8)
        train_data = df.iloc[:train_size].copy()
        test_data = df.iloc[train_size:].copy()
        
        # Máscaras
        mask_train = mask_total.iloc[:train_size].values
        mask_test = mask_total.iloc[train_size:].values
        
        print(f"Dados de treino: {len(train_data)} pontos, {np.sum(mask_train)} válidos ({np.sum(mask_train)/len(mask_train)*100:.1f}%)")
        print(f"Dados de teste: {len(test_data)} pontos, {np.sum(mask_test)} válidos ({np.sum(mask_test)/len(mask_test)*100:.1f}%)")
        
        # 1. Simulação de falhas artificiais
        print("\n1. Simulando falhas artificiais no treino...")
        
        regions = identify_low_missing_regions(train_data['Throughput'].values, mask_train)
        print(f"Regiões com poucos gaps identificadas: {len(regions)}")
        
        artificial_mask, artificial_data = simulate_missing_data(
            train_data['Throughput'].values, mask_train, 
            missing_ratio=0.15, seed=42
        )
        
        original_valid = np.sum(mask_train)
        artificial_valid = np.sum(artificial_mask)
        print(f"Dados válidos: {original_valid} -> {artificial_valid} (removidos: {original_valid - artificial_valid})")
        
        # Aplicar interpolação nos dados com falhas artificiais
        train_data_artificial = train_data.copy()
        train_data_artificial['Throughput'] = artificial_data
        train_data_artificial['Throughput'] = train_data_artificial['Throughput'].interpolate(method='linear')
        
        # 2. Features auxiliares
        print("\n2. Criando features auxiliares...")
        
        mask_feature_train, delta_t_feature_train = create_auxiliary_features(
            train_data_artificial['Throughput'].values, artificial_mask
        )
        
        test_data_interpolated = test_data.copy()
        test_data_interpolated['Throughput'] = test_data_interpolated['Throughput'].interpolate(method='linear')
        
        mask_feature_test, delta_t_feature_test = create_auxiliary_features(
            test_data_interpolated['Throughput'].values, mask_test
        )
        
        print(f"Features auxiliares criadas: mask + delta_t")
        
        # 3. Normalização e dataset com features auxiliares
        print("\n3. Preparando dados com features auxiliares...")
        
        scaler = MinMaxScaler().fit(train_data_artificial[['Throughput']])
        train_scaled = scaler.transform(train_data_artificial[['Throughput']])
        test_scaled = scaler.transform(test_data_interpolated[['Throughput']])
        
        X_train, y_train, mask_y_train = create_dataset_with_auxiliary_features(
            train_scaled.flatten(), mask_feature_train, delta_t_feature_train, look_back=LOOK_BACK
        )
        
        print(f"Dataset de treino: {X_train.shape} (3 features: valor, mask, delta_t)")
        
        # 4. Grid search robusto
        print("\n4. Executando grid search robusto...")
        
        best_params = grid_search_cv_robust(
            create_robust_lstm, 64, X_train, [1e-3, 1e-4],
            y_train, mask_y_train, [100, 200], [32, 64], [10], 'LSTM-Robust'
        )
        
        # 5. Treinamento com loss ponderada
        print("\n5. Treinando modelo com loss ponderada...")
        
        # CORREÇÃO: Criar modelo final com input_shape correto
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = create_robust_lstm(64, input_shape, 5)
        
        # Pesos para treinamento
        sample_weights = np.where(mask_y_train, 1.0, 0.3)
        
        # Early stopping
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=best_params['patience'],
            restore_best_weights=True,
            min_delta=1e-5
        )
        
        # Treinar modelo final
        history = model.fit(
            X_train, y_train,
            epochs=best_params['epochs'],
            batch_size=best_params['batch_size'],
            validation_split=0.2,
            sample_weight=sample_weights,
            callbacks=[early_stop],
            verbose=1
        )
        
        # 6. Predição walk-forward robusta
        print("\n6. Realizando predição walk-forward robusta...")
        
        test_scaled_flat = test_scaled.flatten()
        
        predictions = walk_forward_validation_robust(
            model, scaler, 
            train_scaled.flatten(),
            test_scaled_flat,
            mask_test,
            mask_feature_test,
            delta_t_feature_test,
            look_back=LOOK_BACK,
            window_size=WINDOW_SIZE
        )
        
        # Converter predições para formato original
        predictions = np.array(predictions).reshape(-1, 1)
        predictions_orig = scaler.inverse_transform(predictions)
        
        # 7. Avaliação robusta e salvamento de resultados
        print("\n7. Avaliando e salvando resultados...")
        
        actual_values = test_data['Throughput'].values[LOOK_BACK:]
        mask_valid = mask_test[LOOK_BACK:]
        
        rmse, mae, nrmse, _ = validate_missing_data_prediction_robust(
            predictions_orig, 
            actual_values.reshape(-1, 1), 
            mask_valid,
            substring_desejada
        )
        
        # Salvar métricas
        metrics = {
            'RMSE': float(rmse) if not np.isnan(rmse) else None,
            'MAE': float(mae) if not np.isnan(mae) else None,
            'NRMSE': float(nrmse) if not np.isnan(nrmse) else None,
            'Valid_Points': f"{np.sum(mask_valid)}/{len(mask_valid)}",
            'Valid_Percentage': f"{np.sum(mask_valid)/len(mask_valid)*100:.1f}%"
        }
        
        # Salvar predições
        pred_df = pd.DataFrame({
            'Timestamp': test_data.index[LOOK_BACK:],
            'Actual': actual_values,
            'Predicted': predictions_orig.flatten(),
            'Valid': mask_valid
        })
        pred_file = os.path.join(PREDICTIONS_DIR, f"{substring_desejada}_predictions.csv")
        pred_df.to_csv(pred_file, index=False)
        
        # Salvar modelo
        model_file = os.path.join(MODEL_DIR, f"{substring_desejada}_model.keras")
        model.save(model_file)
        
        # Salvar histórico de treinamento
        history_file = os.path.join(BASE_DIR, f"{substring_desejada}_history.json")
        history_dict = {}
        for key, value in history.history.items():
            history_dict[key] = [float(v) for v in value]
        
        with open(history_file, 'w') as f:
            json.dump(history_dict, f, indent=4)
            
        # Plotar perda de treinamento
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Training History: {substring_desejada}')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True)
        loss_file = os.path.join(PLOTS_LOSS_DIR, f"{substring_desejada}_loss.png")
        plt.savefig(loss_file)
        plt.close()
        
        # Plotar predições vs reais
        plt.figure(figsize=(12, 6))
        valid_indices = np.where(mask_valid)[0]
        plt.plot(pred_df['Timestamp'], pred_df['Actual'], 'b-', label='Actual')
        plt.plot(pred_df['Timestamp'], pred_df['Predicted'], 'r-', alpha=0.7, label='Predicted')
        plt.scatter(
            pred_df.iloc[valid_indices]['Timestamp'],
            pred_df.iloc[valid_indices]['Actual'],
            c='green', s=15, label='Valid Points'
        )
        plt.title(f'Prediction vs Actual: {substring_desejada}')
        plt.xlabel('Timestamp')
        plt.ylabel('Throughput')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        pred_file = os.path.join(PLOTS_PREDICTIONS_DIR, f"{substring_desejada}_prediction.png")
        plt.savefig(pred_file)
        plt.close()
        
        # Registrar métricas no arquivo de saída
        with open(TRAINING_OUTPUT, 'a') as f:
            f.write(f"\n\n{'='*50}\n")
            f.write(f"Resultados para: {substring_desejada}\n")
            f.write(f"RMSE: {rmse:.4f}\n")
            f.write(f"MAE: {mae:.4f}\n")
            f.write(f"NRMSE: {nrmse*100:.2f}%\n")
            f.write(f"Valid Points: {np.sum(mask_valid)}/{len(mask_valid)} ({np.sum(mask_valid)/len(mask_valid)*100:.1f}%)\n")
        
        # Atualizar métricas globais
        global_metrics = {}
        if os.path.exists(METRICS_PATH):
            with open(METRICS_PATH, 'r') as f:
                global_metrics = json.load(f)
                
        global_metrics[substring_desejada] = metrics
        
        with open(METRICS_PATH, 'w') as f:
            json.dump(global_metrics, f, indent=4)
            
        print(f"Processamento completo para {substring_desejada}")
        return metrics
        
    except Exception as e:
        print(f"Erro no processamento de {arquivo}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def main():
    """
    Função principal para executar o pipeline
    """
    arquivos_processados = 0
    
    # Inicializar arquivo de saída
    with open(TRAINING_OUTPUT, 'w') as f:
        f.write("=== INÍCIO DO TREINAMENTO ROBUSTO ===\n")
    
    # Coletar todos os arquivos CSV
    arquivos = []
    for root, dirs, files in os.walk(THROUGHPUT_DATASETS):
        for file in files:
            if file.endswith('.csv'):
                arquivos.append((os.path.join(root, file), file))
    
    # Processar cada arquivo
    for caminho, arquivo in arquivos:
        resultado = processo_arquivo_robusto(caminho, arquivo)
        if resultado:
            arquivos_processados += 1
            
    print(f"\n{'='*50}")
    print(f"Processamento completo!")
    print(f"Arquivos processados: {arquivos_processados}/{len(arquivos)}")
    print(f"Resultados salvos em: {BASE_DIR}")
    print(f"{'='*50}")

if __name__ == "__main__":
    main()

2025-07-03 19:24:41.865577: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 19:24:41.865972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-03 19:24:41.867794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-03 19:24:41.872758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751581481.881244 3199140 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751581481.88


Processando: test-failure - treated bbr esmond data ap-ba 07-03-2023.csv
Dados de treino: 579 pontos, 545 válidos (94.1%)
Dados de teste: 145 pontos, 78 válidos (53.8%)

1. Simulando falhas artificiais no treino...
Regiões com poucos gaps identificadas: 17
Dados válidos: 545 -> 440 (removidos: 105)

2. Criando features auxiliares...
Features auxiliares criadas: mask + delta_t

3. Preparando dados com features auxiliares...
Dataset de treino: (576, 3, 3) (3 features: valor, mask, delta_t)

4. Executando grid search robusto...
Erro no processamento de treated bbr esmond data ap-ba 07-03-2023.csv: Argument `learning_rate` should be float, or an instance of LearningRateSchedule, or a callable (that takes in the current iteration value and returns the corresponding learning rate value). Received instead: learning_rate=5

Processamento completo!
Arquivos processados: 0/1
Resultados salvos em: resultados-recursive-prediction-robust


E0000 00:00:1751581482.621473 3199140 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1751581482.621668 3199140 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/clara/Documentos/recursive-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Traceback (most recent call last):
  File "/tmp/ipykernel_3199140/70487149